In [6]:
import recordlinkage as rl
import pandas as pd
import os
import random as ran
from sklearn.model_selection import GridSearchCV

In [7]:
schema_path = "../schema_matching/csv/schema_final.csv"

In [8]:
schema = pd.read_csv(schema_path, index_col=0)
schema = schema.reset_index(drop=True) # Per evitare che gli indici siano letti come decimanli (i.e. 1.0 invece di 1)

/var/folders/zq/pjr33nrn6175lz95s2t31wz00000gn/T/ipykernel_9401/2791656242.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  schema = pd.read_csv(schema_path, index_col=0)


# Preprocessamento

In [9]:
from recordlinkage.preprocessing import *

## Cleaning dei dati ...

In [26]:
schema_cleaned = schema # creo una copia per non modificare lo schema iniziale
schema_cleaned_path = "./csv/schema_cleaned.csv"

In [27]:
# Cleaning dei dati iterando per riga
for i in range(len(schema.index)):
    row = schema.iloc[i]
    row_series = row.squeeze() # BOO
    nome = pd.Series(row["name"])
    try:
        nome_cleaned = clean(nome, lowercase=True, replace_by_none='\n', replace_by_whitespace='\ {2,}', strip_accents='unicode', remove_brackets=False, encoding='utf-8', decode_error='strict')
    except:
        print(nome)
    # Si evita il cleaning di caratteri relative alle valute presenti nei datasets
    row_series_cleaned = clean(row_series, lowercase=True, replace_by_none=r'[^ \\.\\-\\_A-Za-z0-9$€£¥₩₽₹฿₪¤¢₺₱]+', replace_by_whitespace=r'[\\-\\_]', strip_accents=None, remove_brackets=False, encoding='utf-8', decode_error='strict')
    schema_cleaned.iloc[i] = row_series_cleaned
    schema_cleaned.iloc[i]["name"] = nome_cleaned[0]

In [28]:
schema_cleaned.to_csv(schema_cleaned_path, index_label="id")

# Occorrenze token per riga

In [12]:
# value_occurence(schema)

# Blocking

In [13]:
indexer = rl.Index()
indexer.block(left_on='name', right_on='name')
candidate_links = indexer.index(schema_cleaned)
print(candidate_links)

MultiIndex([( 49851,      0),
            ( 49852,      1),
            ( 41924,      2),
            ( 49853,      2),
            ( 49853,  41924),
            ( 67141,      2),
            ( 67141,  41924),
            ( 67141,  49853),
            (122147,      2),
            (122147,  41924),
            ...
            (160451, 146351),
            (160472, 146373),
            (160175, 146391),
            (160370, 146476),
            (160323, 146484),
            (146641, 146518),
            (159761, 146625),
            (146824, 146629),
            (146727, 146649),
            (160411, 160410)],
           length=549201)


# Labeling

In [14]:
keys = ['name', 'country', 'market cap', 'founded year', 'employees', 'industry', 'sector',
     'ceo', 'revenue', 'stock', 'share price', 'city', 'address', 'website']

clear = lambda: os.system('clear')

In [15]:
n_sampled_matches = 450
choices = ran.choices(candidate_links, k=n_sampled_matches)

lKeys = {key : f"l_{key}" for key in keys}
rKeys = {key : f"r_{key}" for key in keys}

idxKeys = ["id_1", "id_2"]

choices_column = [*idxKeys, *lKeys.values(), *rKeys.values()] # id_1, id_2, l_attr, r_attr
choices_df = pd.DataFrame(columns=choices_column) # 
for choice in choices:
    lRow = schema_cleaned.iloc[[choice[0]]]
    rRow = schema_cleaned.iloc[[choice[1]]]

    lRow = lRow.rename(columns=lKeys).reset_index(drop=True)
    rRow = rRow.rename(columns=rKeys).reset_index(drop=True)

    idxs = pd.DataFrame({idxKeys[0]: [choice[0]], idxKeys[1]: [choice[1]]})
    row = pd.concat([idxs, lRow, rRow], axis=1)
    choices_df = pd.concat([choices_df, row], axis=0)

# choices_df.reset_index(inplace=True)
choices_df.set_index(idxKeys)

,,l_name,l_country,l_market cap,l_founded year,l_employees,l_industry,l_sector,l_ceo,l_revenue,l_stock,...,r_employees,r_industry,r_sector,r_ceo,r_revenue,r_stock,r_share price,r_city,r_address,r_website
id_1,id_2,,,,,,,,,,,,,,,,,,,,,
183410,23732,granite construction,usa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$35.92,NaN,NaN,NaN
106560,37823,ao world,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$0.83,NaN,NaN,NaN
156347,105006,birchcliff energy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$6.89,NaN,NaN,NaN
35712,24186,brandywine realty trust,usa,$1.13 b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$6.41,NaN,NaN,NaN
132050,121051,aker bp asa,norway,9.343 billion usd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,energy oil and gas petroleum,NaN,not found,5639 million usd,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177864,46897,cyient,india,$1.18 b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155564,36322,nongshim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$277.23,NaN,NaN,NaN
63221,27054,bce,NaN,NaN,1970.0,NaN,telecommunications services,NaN,NaN,$18.7b,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
choices_path = "./csv/choices.csv"
choices_df.to_csv(choices_path, index=None)

In [17]:
choices_path_gallo = "./csv/gallo.csv"
choices_path_gatto = "./csv/gatto.csv"
choices_path_moli = "./csv/moli.csv"

sampled_matches_per_person = n_sampled_matches//3

choices_df_gallo = choices_df.iloc[:sampled_matches_per_person]
print(len(choices_df_gallo))

choices_df_gatto = choices_df.iloc[sampled_matches_per_person:2*sampled_matches_per_person]
print(len(choices_df_gatto))

choices_df_moli = choices_df.iloc[2*sampled_matches_per_person:]
print(len(choices_df_moli))

choices_df_gallo.to_csv(choices_path_gallo, index=None)
choices_df_gatto.to_csv(choices_path_gatto, index=None)
choices_df_moli.to_csv(choices_path_moli, index=None)

150
150
150


# Comparing

In [37]:
compare_cl = rl.Compare()
compare_cl.string("name", "name", threshold=0.85, label="name")
compare_cl.string("country", "country", label="country")
compare_cl.string("founded year", "founded year", label="founded year")
compare_cl.string("industry", "industry", label="industry")
compare_cl.string("sector", "sector", label="sector")
compare_cl.string("address", "address", label="address")
compare_cl.string("city", "city", label="city")
compare_cl.string("ceo", "ceo", label="ceo")
features = compare_cl.compute(candidate_links, schema, schema)

In [38]:
features

name  country  founded year  industry  sector  address  city  \
49851  0        1.0      1.0           0.0       1.0     0.0      0.0   1.0   
49852  1        1.0      1.0           0.0       1.0     0.0      0.0   1.0   
41924  2        1.0      0.0           0.0       0.0     0.0      0.0   0.0   
49853  2        1.0      1.0           0.0       1.0     0.0      0.0   1.0   
       41924    1.0      0.0           0.0       0.0     0.0      0.0   0.0   
...             ...      ...           ...       ...     ...      ...   ...   
146641 146518   1.0      0.0           1.0       0.0     0.0      0.0   0.0   
159761 146625   1.0      0.0           1.0       0.0     0.0      0.0   0.0   
146824 146629   1.0      0.0           1.0       1.0     0.0      0.0   0.0   
146727 146649   1.0      0.0           1.0       1.0     0.0      0.0   0.0   
160411 160410   1.0      0.0           1.0       1.0     0.0      0.0   0.0   

               ceo  
49851  0       0.0  
49852  1       0.0  
41924  2       0.0  
49853  2       0.0  
       41924   0.0  
...            ...  
146641 146518  0.0  
159761 146625  0.0  
146824 146629  0.0  
146727 146649  0.0  
160411 160410  0.0  

[549201 rows x 8 columns]

In [39]:
features.describe()

,name,country,founded year,industry,sector,address,city,ceo
count,549201.0,549201.000000,549201.000000,549201.000000,549201.000000,549201.000000,549201.00000,549201.000000
mean,1.0,0.428898,0.042811,0.035205,0.042350,0.011631,0.00485,0.049379
std,0.0,0.487790,0.193212,0.175675,0.200949,0.088956,0.06944,0.203537
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [40]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.000000       314
5.972222         1
5.968750         1
5.966102         1
5.961538         1
             ...  
1.017857         1
1.016667         1
1.016393         1
1.011820         1
1.000000    245262
Length: 5788, dtype: int64

In [41]:
record_linked = features[features.sum(axis=1) > 1]

In [42]:
print(record_linked)

               name  country  founded year  industry  sector  address  city  \
49851  0        1.0      1.0           0.0       1.0     0.0      0.0   1.0   
49852  1        1.0      1.0           0.0       1.0     0.0      0.0   1.0   
41924  2        1.0      0.0           0.0       0.0     0.0      0.0   0.0   
49853  2        1.0      1.0           0.0       1.0     0.0      0.0   1.0   
       41924    1.0      0.0           0.0       0.0     0.0      0.0   0.0   
...             ...      ...           ...       ...     ...      ...   ...   
146641 146518   1.0      0.0           1.0       0.0     0.0      0.0   0.0   
159761 146625   1.0      0.0           1.0       0.0     0.0      0.0   0.0   
146824 146629   1.0      0.0           1.0       1.0     0.0      0.0   0.0   
146727 146649   1.0      0.0           1.0       1.0     0.0      0.0   0.0   
160411 160410   1.0      0.0           1.0       1.0     0.0      0.0   0.0   

               ceo  
49851  0       0.0  
49852  1 

# Caricamento dei dataset per classificazione

In [24]:
golden_path_gallo = "./csv/golden_links_gallo.csv"
golden_path_gatto = "./csv/golden_links_gatto.csv"
golden_path_moli = "./csv/golden_links_moli.csv"

golden_gallo = pd.read_csv(golden_path_gallo, index_col=None)
golden_gatto = pd.read_csv(golden_path_gatto, index_col=None)
golden_moli = pd.read_csv(golden_path_moli, index_col=None)

golden = pd.concat([golden_gallo, golden_gatto, golden_moli], axis=0)

golden_path = "./csv/golden_links.csv"

golden = golden.reset_index(drop=True)
golden.to_csv(golden_path, index=None)

FileNotFoundError: [Errno 2] No such file or directory: './csv/golden_links_gallo.csv'

In [ ]:
golden = pd.read_csv(golden_path, index_col=[0,1])
golden_links = golden[golden["label"]==1].index

5

# Classification (Unsupervised)

In [ ]:
ECM = rl.ECMClassifier(binarize=0.6)
result = ECM.fit_predict(record_linked)
print("Possible record linked size:", len(record_linked))
print("Record linked size:", len(result))
print(f"Removed {len(record_linked) - len(result)} possible links")

Possible record linked size: 303932
Record linked size: 27940
Removed 275992 possible links


# Evaluation (Unsupervised)

### Confusion matrix

In [ ]:
rl.confusion_matrix(golden_links, result)

### Precision

In [ ]:
rl.precision(golden_links, result)

### Recall

In [ ]:
rl.recall(golden_links, result)

### Accuracy

In [ ]:
rl.accuracy(golden_links, result)

### F-score

In [ ]:
rl.fscore(golden_links, result)

### Specificity

In [ ]:
rl.specificity(golden_links, result)

# Classification (Supervised)

In [ ]:
rlSvm = rl.SVMClassifier()
params = {}
gridSearch = GridSearchCV(estimator=rlSvm, param_grid=params)

# Evaluation (Supervised)

### Confusion matrix

In [ ]:
rl.confusion_matrix(golden_links, )

### Precision

In [ ]:
rl.precision(golden_links,)

### Recall

In [ ]:
rl.recall(golden_links,)

### Accuracy

In [ ]:
rl.accuracy(golden_links,)

### F-score

In [ ]:
rl.fscore(golden_links,)

### Specificity

In [ ]:
rl.specificity(golden_links,)